# SQL. Анализ сервиса книг по подписке

## Описание проекта

**Цель анализа** - проанализировать базу данных, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Проведенный анализ должен сформулировать ценностное предложение для нового продукта.

**Задачи**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица books**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подключение к базе данных и изучение данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

def show(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

In [2]:
query_books = '''
        SELECT * FROM books
        '''
books = show(query_books)

query_authors = '''
                SELECT * FROM authors
                '''
authors = show(query_authors)

query_publishers = '''
                   SELECT * FROM publishers
                   '''
publishers = show(query_publishers)

query_ratings = '''
                SELECT * FROM ratings
                '''
ratings = show(query_ratings)

query_reviews = '''
                SELECT * FROM reviews
                '''
reviews = show(query_reviews)

In [3]:
display(books.head())
books.info()
books.isna().sum()
books.duplicated().sum()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


0

In [4]:
display(authors.head())
authors.info()
authors.isna().sum()
authors.duplicated().sum()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


0

In [5]:
display(publishers.head())
publishers.info()
publishers.isna().sum()
publishers.duplicated().sum()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


0

In [6]:
display(ratings.head())
ratings.info()
ratings.isna().sum()
ratings.duplicated().sum()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


0

In [7]:
display(reviews.head())
reviews.info()
reviews.isna().sum()
reviews.duplicated().sum()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


0

# Вывод

При первичном изучении данных пропущенных значений и дубликатов обнаружено не было, в таблице books столбец publication_date можно было бы привести к нужному типу данных, но на этом этапе в этом нет необходимости. Для анализа БД имеется пять таблиц - данные о книгах, изданиях, авторах, оценках и обзорах пользователей. Данные полностью готову к последующему анализу.

##  Количество книг, вышедших после 1 января 2000 года

In [8]:
count_book = '''
             SELECT COUNT(book_id) AS count_book
             FROM books
             WHERE publication_date > '2000-01-01'
            '''
request = pd.io.sql.read_sql(count_book, con = engine)

print("После 1 января 2000 года вышло", request.iloc[0,0], "книг")

После 1 января 2000 года вышло 819 книг


# Вывод

По полученным результатам выяснилось, что после 1 января 2000 года было выпущено 819 книг.

## Количество обзоров и средняя оценка для каждой книги

In [15]:
review_rating = '''
                SELECT b.book_id AS book_id,
                       b.title AS title, 
                       b.publication_date AS date,
                       COUNT(DISTINCT re.review_id) AS count_review,
                       AVG(r.rating) AS avg_rating
                FROM books AS b
                LEFT JOIN ratings AS r ON r.book_id = b.book_id
                LEFT JOIN reviews AS re ON re.book_id = b.book_id
                GROUP BY b.title, b.book_id
                ORDER BY count_review DESC
                LIMIT 10
            '''
request = pd.io.sql.read_sql(review_rating, con = engine)

request

,book_id,title,date,count_review,avg_rating
0,948,Twilight (Twilight #1),2006-09-06,7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),2006-03-28,6,3.830508
2,207,Eat Pray Love,2007-02-01,6,3.395833
3,627,The Alchemist,1993-05-01,6,3.789474
4,673,The Catcher in the Rye,2001-01-30,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,2004-05-18,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,2004-05-01,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,1999-06-02,6,4.287500
8,497,Outlander (Outlander #1),2005-07-26,6,4.125000
9,656,The Book Thief,2006-03-14,6,4.264151


In [10]:
review_rating = '''
                SELECT b.book_id AS book_id,
                       b.title AS title, 
                       b.publication_date AS date,
                       COUNT(DISTINCT re.review_id) AS count_review,
                       AVG(r.rating) AS avg_rating
                FROM books AS b
                LEFT JOIN ratings AS r ON r.book_id = b.book_id
                LEFT JOIN reviews AS re ON re.book_id = b.book_id
                GROUP BY b.title, b.book_id
                ORDER BY avg_rating DESC
                LIMIT 10
            '''
request = pd.io.sql.read_sql(review_rating, con = engine)

request

,book_id,title,date,count_review,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),1987-03-03,2,5.0
1,169,Crucial Conversations: Tools for Talking When ...,2002-07-09,2,5.0
2,62,Alas Babylon,2005-07-05,2,5.0
3,57,Act of Treason (Mitch Rapp #9),2006-10-10,2,5.0
4,76,Angels Fall,2006-07-11,2,5.0
5,136,Captivating: Unveiling the Mystery of a Woman'...,2005-04-05,2,5.0
6,55,A Woman of Substance (Emma Harte Saga #1),2005-12-01,2,5.0
7,20,A Fistful of Charms (The Hollows #4),2006-06-27,2,5.0
8,17,A Dirty Job (Grim Reaper #1),2006-03-21,4,5.0
9,182,Dead Souls,2004-07-29,2,5.0


# Вывод

**По количеству обзоров**

По полученным результатам выяснилось, что по количеству обзоров лидирует книга Twilight (Twilight #1) 2006-09-06 года выпуска, количнство обзоров рано 7, а средняя оценка 3.7. На втором месте книга The Da Vinci Code (Robert Langdon #2)	2006-03-28 года выпуска: количество обзоров - 6, средняя оценка - 3.8. На третьем месте Eat Pray Love 2007-02-01 годы выпуска: количество обзоров - 6, средняя оценка - 3.4.

**Средняя оценка**

Топ-3 книги:

- Arrows of the Queen (Heralds of Valdemar #1), год выпуска - 1987-03-03, кол-во обзоров - 2, средняя оценка - 5.0
- Crucial Conversations: Tools for Talking When ..., год выпуска - 2002-07-09, кол-во обзоров - 2, средняя оценка - 5.0
- Alas Babylon, год выпуска	- 2005-07-05, кол-во обзоров - 2, средняя оценка - 5.0

Стоит также отметить книгу:
- A Dirty Job (Grim Reaper #1), год выпуска	- 2006-03-21, кол-во обзоров - 4, средняя оценка - 5.0
Это очень хороший показатель сотношения количества обзоров и средней оценки.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц - исключим из анализа брошюры

In [11]:
publisher_famous = '''
                SELECT p.publisher_id AS pub_id,
                       p.publisher AS pub_title, 
                       COUNT(b.book_id) AS count_books
                FROM publishers AS p
                LEFT JOIN books AS b ON b.publisher_id = p.publisher_id
                WHERE b.num_pages > 50
                GROUP BY pub_title, pub_id
                ORDER BY count_books DESC
                LIMIT 1
            '''
request = pd.io.sql.read_sql(publisher_famous, con = engine)

request

,pub_id,pub_title,count_books
0,212,Penguin Books,42


# Вывод

Наибольшее количество книг - 42 книги, выпустило издательство Penguin Books. Возможно оно пользуется популярностью и известностью среди авторов, поэтому они хотят сотрудничать именно с этим издательством.

## Автор с самой высокой средней оценкой книг 

In [12]:
author_rating = '''
                SELECT 
                    res.author AS author,  
                    AVG(res.avg_rating) AS rating 
                FROM (SELECT
                        authors.author AS author,
                        b.book_id AS id, 
                        AVG(r.rating) AS avg_rating,
                        COUNT(r.rating) AS rating_count
                    FROM authors
                        INNER JOIN books AS b ON b.author_id = authors.author_id
                        INNER JOIN ratings AS r ON r.book_id = b.book_id
                    GROUP BY author,id
                        HAVING
                        COUNT(r.rating) > 50) AS res
                GROUP BY author
                ORDER BY rating DESC
                LIMIT 10
                '''
request = pd.io.sql.read_sql(author_rating, con = engine)

request

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


# Вывод

Автором книги, получивший наибольшую среднюю оценку своих работ - J.K. Rowling/Mary GrandPré.

Топ-5 авторов, получивших наибольшую среднюю оценку:
- J.K. Rowling/Mary GrandPré - 4.28
- Markus Zusak/Cao Xuân Việt Khương - 4.26
- J.R.R. Tolkien - 4.26
- Louisa May Alcott	- 4.19
- Rick Riordan - 4.08


## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
avg_reviews = '''
               SELECT 
                   ROUND(AVG(res.count_review), 2)
               FROM(
                   SELECT
                       username,
                       COUNT(review_id) AS count_review
                   FROM
                       reviews
                   WHERE 
                       username IN (
                           SELECT
                               username
                           FROM
                               ratings
                           GROUP BY
                               username 
                           HAVING 
                           COUNT(ratings.rating_id) >= 50)
                   GROUP BY
                       username) AS res  
                '''
request = pd.io.sql.read_sql(avg_reviews, con = engine)

request                           
              

,round
0,24.22


# Вывод

По полученным результатам выяснилось, что пользователи, поставившие более 50 оценок написали в среднем 24 обзора.

## Общий вывод

При анализе базы данных сервиса книг по подписке получили следующие результаты:

- После 1 января 2000 года вышло 819 книг
-  **По количеству обзоров** лидирует книга Twilight (Twilight #1) 2006-09-06 года выпуска, количнство обзоров рано 7, а средняя оценка 3.7. На втором месте книга The Da Vinci Code (Robert Langdon #2) 2006-03-28 года выпуска: количество обзоров - 6, средняя оценка - 3.8. На третьем месте Eat Pray Love 2007-02-01 годы выпуска: количество обзоров - 6, средняя оценка - 3.4.
- **Топ-3 книги по средней оценке**:

Arrows of the Queen (Heralds of Valdemar #1), год выпуска - 1987-03-03, кол-во обзоров - 2, средняя оценка - 5.0
Crucial Conversations: Tools for Talking When ..., год выпуска - 2002-07-09, кол-во обзоров - 2, средняя оценка - 5.0
Alas Babylon, год выпуска - 2005-07-05, кол-во обзоров - 2, средняя оценка - 5.0

- Стоит также отметить книгу:

A Dirty Job (Grim Reaper #1), год выпуска - 2006-03-21, кол-во обзоров - 4, средняя оценка - 5.0 Это очень хороший показатель сотношения количества обзоров и средней оценки.
- Наибольшее количество книг - 42 книги, выпустило издательство Penguin Books. Возможно оно пользуется популярностью и известностью среди авторов, поэтому они хотят сотрудничать именно с этим издательством.

- Пользователи, поставившие более 50 оценок написали в среднем 24 обзора.
